In [72]:
import pandas as pd

# Set display options to show more rows and columns
pd.set_option('display.max_columns', None) # Show all columns

listings = pd.read_csv('../AirBnb datasets/barcelona/listings.csv')

listings.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,269467,https://www.airbnb.com/rooms/269467,20240615011618,2024-06-15,city scrape,Sunny and quiet Loft next to Sagrada Familia,The apartment is a cosy open space with 65 m2....,NaN,https://a0.muscache.com/pictures/a6445a90-94eb...,1411775,https://www.airbnb.com/users/show/1411775,Jeremies,2011-11-16,"Barcelona, Spain","We are Ok Apartment Barcelona, a local agency ...",within an hour,100%,88%,f,https://a0.muscache.com/im/pictures/user/1f974...,https://a0.muscache.com/im/pictures/user/1f974...,La Sagrada Família,5.0,7.0,"['email', 'phone']",t,t,NaN,la Sagrada Família,Eixample,41.401810,2.17610,Entire loft,Entire home/apt,2,1.0,1 bath,1.0,1.0,"[""Kitchen"", ""Patio or balcony"", ""Host greets y...",$176.00,7,760,7,7,760,760,7.0,760.0,NaN,t,0,0,0,257,2024-06-15,90,17,1,2018-01-01,2024-05-22,4.74,4.87,4.59,4.83,4.79,4.81,4.67,HUTB-001129,t,3,3,0,0,1.15
1,270197,https://www.airbnb.com/rooms/270197,20240615011618,2024-06-15,city scrape,*monthly stays* 4 Bedroom Apartment near Rambla,Rented only for monthly periods. 4 bedroom apa...,NaN,https://a0.muscache.com/pictures/69907440-d8df...,1414702,https://www.airbnb.com/users/show/1414702,Ferran,2011-11-16,"Barcelona, Spain","Hello, everyone!\n\nBorn and bred in Barcelona...",within an hour,98%,97%,f,https://a0.muscache.com/im/pictures/user/User-...,https://a0.muscache.com/im/pictures/user/User-...,El Raval,36.0,112.0,"['email', 'phone']",t,t,NaN,el Raval,Ciutat Vella,41.379360,2.17376,Entire rental unit,Entire home/apt,6,1.5,1.5 baths,4.0,5.0,"[""Kitchen"", ""Washer"", ""Smoking allowed"", ""Wifi...",$74.00,32,760,32,32,760,760,32.0,760.0,NaN,t,0,0,0,0,2024-06-15,2,0,0,2016-07-03,2016-08-13,4.00,5.00,4.50,5.00,5.00,5.00,4.50,N/A SOLO ESTANCIAS MENSUALES,f,27,27,0,0,0.02
2,18674,https://www.airbnb.com/rooms/18674,20240615011618,2024-06-15,city scrape,Huge flat for 8 people close to Sagrada Familia,110m2 apartment to rent in Barcelona. Located ...,Apartment in Barcelona located in the heart of...,https://a0.muscache.com/pictures/13031453/413c...,71615,https://www.airbnb.com/users/show/71615,Mireia Maria,2010-01-19,"Barcelona, Spain","We are Mireia (47) & Maria (49), two multiling...",within an hour,95%,85%,f,https://a0.muscache.com/im/pictures/user/User-...,https://a0.muscache.com/im/pictures/user/User-...,la Sagrada Família,41.0,45.0,"['email', 'phone']",t,t,"Barcelona, CT, Spain",la Sagrada Família,Eixample,41.405560,2.17262,Entire rental unit,Entire home/apt,8,2.0,2 baths,3.0,6.0,"[""Pack \u2019n play/Travel crib"", ""Free street...",$257.00,1,1125,1,4,1125,1125,2.1,1125.0,NaN,t,10,36,66,177,2024-06-15,41,5,0,2013-05-27,2024-05-02,4.35,4.45,4.63,4.75,4.68,4.78,4.30,HUTB-002062,t,27,27,

In [73]:
# Identify columns where all values are the same
constant_columns = [col for col in listings.columns if listings[col].nunique() == 1]

# Print the columns where all values are the same
print("Columns where all values are the same:", constant_columns)

Columns where all values are the same: ['scrape_id', 'last_scraped', 'calendar_last_scraped']


In [74]:
# Remove columns where all values are the same
columns_to_remove = ['scrape_id', 'last_scraped', 'calendar_last_scraped']
listings = listings.drop(columns=columns_to_remove)

In [75]:
# Remove redundant or uninformative features
columns_to_remove = ['listing_url', 'source', 'name', 'description', 'neighborhood_overview',
                     'picture_url', 'host_id', 'host_url', 'host_name', 'host_about', 'host_thumbnail_url',
                     'host_picture_url', 'license', 'calendar_updated', 'host_location', 'host_neighbourhood',
                     'host_total_listings_count', 'host_verifications', 'neighbourhood', 
                     'neighbourhood_cleansed', 'bathrooms_text', 'minimum_minimum_nights', 
                     'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights', 
                     'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'has_availability', 'availability_30', 
                     'availability_60', 'availability_90', 'availability_365', 'calculated_host_listings_count',
                     'calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_private_rooms', 
                     'calculated_host_listings_count_shared_rooms', 'latitude', 'longitude']

listings = listings.drop(columns=columns_to_remove)

In [76]:
# Convert boolean values into binary representations
listings['host_is_superhost'] = listings['host_is_superhost'].replace({'t': 1, 'f': 0})
listings['instant_bookable'] = listings['instant_bookable'].replace({'t': 1, 'f': 0})
listings['host_has_profile_pic'] = listings['host_has_profile_pic'].replace({'t': 1, 'f': 0})
listings['host_identity_verified'] = listings['host_identity_verified'].replace({'t': 1, 'f': 0})

/var/folders/hr/dg_n9mds23z57wvq3v_cd7340000gn/T/ipykernel_4835/365010975.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  listings['host_is_superhost'] = listings['host_is_superhost'].replace({'t': 1, 'f': 0})
/var/folders/hr/dg_n9mds23z57wvq3v_cd7340000gn/T/ipykernel_4835/365010975.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  listings['instant_bookable'] = listings['instant_bookable'].replace({'t': 1, 'f': 0})
/var/folders/hr/dg_n9mds23z57wvq3v_cd7340000gn/T/ipykernel_4835/365010975.py:4: FutureWarning: Downcasting beh

In [77]:
# Remove the dollar symbol and commas, then convert the column to numeric
listings['price'] = listings['price'].str.replace('$', '', regex=False).str.replace(',', '', regex=False).astype(float)

listings['host_acceptance_rate'] = listings['host_acceptance_rate'].str.replace('%', '', regex=False).astype(float)
listings['host_response_rate'] = listings['host_response_rate'].str.replace('%', '', regex=False).astype(float)

In [78]:
# Handle Date columns (convert to datetime, then to numeric)
if 'host_since' in listings.columns:  # Replace 'date_column_name' with your actual date column name
    listings['host_since'] = pd.to_datetime(listings['host_since'])
    listings['host_since'] = listings['host_since'].astype(int) // 10**9  # Convert to Unix timestamp

if 'first_review' in listings.columns:  # Replace 'date_column_name' with your actual date column name
    listings['first_review'] = pd.to_datetime(listings['first_review'])
    listings['first_review'] = listings['first_review'].astype(int) // 10**9  # Convert to Unix timestamp

if 'last_review' in listings.columns:  # Replace 'date_column_name' with your actual date column name
    listings['last_review'] = pd.to_datetime(listings['last_review'])
    listings['last_review'] = listings['last_review'].astype(int) // 10**9  # Convert to Unix timestamp

In [79]:
listings

,id,host_since,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_has_profile_pic,host_identity_verified,neighbourhood_group_cleansed,property_type,room_type,accommodates,bathrooms,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,reviews_per_month
0,269467,1321401600,within an hour,100.0,88.0,0.0,5.0,1.0,1.0,Eixample,Entire loft,Entire home/apt,2,1.0,1.0,1.0,"[""Kitchen"", ""Patio or balcony"", ""Host greets y...",176.0,7,760,90,17,1,1514764800,1716336000,4.74,4.87,4.59,4.83,4.79,4.81,4.67,1,1.15
1,270197,1321401600,within an hour,98.0,97.0,0.0,36.0,1.0,1.0,Ciutat Vella,Entire rental unit,Entire home/apt,6,1.5,4.0,5.0,"[""Kitchen"", ""Washer"", ""Smoking allowed"", ""Wifi...",74.0,32,760,2,0,0,1467504000,1471046400,4.00,5.00,4.50,5.00,5.00,5.00,4.50,0,0.02
2,18674,1263859200,within an hour,95.0,85.0,0.0,41.0,1.0,1.0,Eixample,Entire rental unit,Entire home/apt,8,2.0,3.0,6.0,"[""Pack \u2019n play/Travel crib"", ""Free street...",257.0,1,1125,41,5,0,1369612800,1714608000,4.35,4.45,4.63,4.75,4.68,4.78,4.30,1,0.30
3,272282,1321747200,NaN,NaN,98.0,1.0,2.0,1.0,1.0,Eixample,Entire rental unit,Entire home/apt,8,NaN,3.0,NaN,"[""Pack \u2019n play/Travel crib"", ""Kitchen"", ""...",NaN,4,1125,414,32,0,1341273600,1710720000,4.92,4.93,4.97,4.94,4.96,4.84,4.80,0,2.84
4,23197,1268092800,within an hour,100.0,87.0,0.0,6.0,1.0,1.0,Sant Martí,Entire rental unit,Entire home/apt,5,2.0,3.0,4.0,"[""Free street parking"", ""Neutral body soap"", ""...",249.0,3,32,78,7,1,1300147200,1716163200,4.79,4.93,4.89,4.95,4.99,4.62,4.64,0,0.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18920,1173891409488923931,1460073600,within an hour,100.0,48.0,0.0,53.0,1.0,1.0,Ciutat Vella,Entire rental unit,Entire home/apt,2,1.0,1.0,1.0,"[""Kitchen"", ""Washer"", ""Wifi"", ""Air conditionin...",86.0,32,345,0,0,0,-9223372037,-9223372037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
18921,1178741129427938140,1589500800,within a few hours,99.0,98.0,0.0,702.0,1.0,1.0,Ciutat Vella,Entire rental unit,Entire home/apt,4,2.0,2.0,2.0,"[""Cleaning products"", ""Kitchen"", ""Dryer"", ""Fir...",167.0,31,330,0,0,0,-9223372037,-9223372037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
18922,1178910876458935142,1589500800,within a few hours,99.0,98.0,0.0,702.0,1.0,1.0,Ciutat Vella,Entire rental unit,Entire home/apt,4,2.0,2.0,2.0,"[""Cleaning products"", ""Kitchen"", ""Dryer"", ""Fir...",165.0,31,330,0,0,0,-9223372037,-9223372037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
18923,1173984319228839137,1568246400,within an hour,99.0,99.0,0.0,24.0,1.0,1.0,Ciutat Vella,Private room in rental unit,Private room,2,1.0,1.0,1.0,"[""Wifi"", ""Lock on bedroom door""]",44.0,5,90,0,0,0,-9223372037,-9223372037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN


<h3>Importing amenities after being preprocessed</h3>

In [80]:
amenities_final = pd.read_csv('amenities.csv')

amenities_final

,wifi,heating,essentials,kitchen,smoke_alarm,air_conditioning,hangers,coffee_maker,dedicated_workspace,hot_water_kettle,hair_dryer,iron,tv,hot_water,washer,dryer,first_aid_kit,fire_extinguisher,refrigerator,pool,Gym,pets_allowed,free_parking_on_premises,paid_parking_on_premises,self_check-in
0,1,1,1,1,0,0,1,1,1,0,1,1,1,1,1,0,0,0,1,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,0,0,1,1,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,1,0
3,1,1,1,1,1,1,1,0,0,1,1,1,0,1,1,0,0,1,1,0,0,1,0,0,0
4,1,0,1,1,1,0,1,0,1,1,1,1,0,1,0,0,0,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18920,1,0,0,1,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
18921,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,0,1,0,0,0
18922,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,0,1,0,0,0
18923,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [81]:
# Concatenate the original DataFrame (excluding the old 'amenities' column) with the new boolean features
listings = pd.concat([listings.drop(columns=['amenities']), amenities_final], axis=1)

listings

,id,host_since,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_has_profile_pic,host_identity_verified,neighbourhood_group_cleansed,property_type,room_type,accommodates,bathrooms,bedrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,reviews_per_month,wifi,heating,essentials,kitchen,smoke_alarm,air_conditioning,hangers,coffee_maker,dedicated_workspace,hot_water_kettle,hair_dryer,iron,tv,hot_water,washer,dryer,first_aid_kit,fire_extinguisher,refrigerator,pool,Gym,pets_allowed,free_parking_on_premises,paid_parking_on_premises,self_check-in
0,269467,1321401600,within an hour,100.0,88.0,0.0,5.0,1.0,1.0,Eixample,Entire loft,Entire home/apt,2,1.0,1.0,1.0,176.0,7,760,90,17,1,1514764800,1716336000,4.74,4.87,4.59,4.83,4.79,4.81,4.67,1,1.15,1,1,1,1,0,0,1,1,1,0,1,1,1,1,1,0,0,0,1,0,0,0,0,0,0
1,270197,1321401600,within an hour,98.0,97.0,0.0,36.0,1.0,1.0,Ciutat Vella,Entire rental unit,Entire home/apt,6,1.5,4.0,5.0,74.0,32,760,2,0,0,1467504000,1471046400,4.00,5.00,4.50,5.00,5.00,5.00,4.50,0,0.02,1,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
2,18674,1263859200,within an hour,95.0,85.0,0.0,41.0,1.0,1.0,Eixample,Entire rental unit,Entire home/apt,8,2.0,3.0,6.0,257.0,1,1125,41,5,0,1369612800,1714608000,4.35,4.45,4.63,4.75,4.68,4.78,4.30,1,0.30,1,1,1,1,0,0,1,1,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,1,0
3,272282,1321747200,NaN,NaN,98.0,1.0,2.0,1.0,1.0,Eixample,Entire rental unit,Entire home/apt,8,NaN,3.0,NaN,NaN,4,1125,414,32,0,1341273600,1710720000,4.92,4.93,4.97,4.94,4.96,4.84,4.80,0,2.84,1,1,1,1,1,1,1,0,0,1,1,1,0,1,1,0,0,1,1,0,0,1,0,0,0
4,23197,1268092800,within an hour,100.0,87.0,0.0,6.0,1.0,1.0,Sant Martí,Entire rental unit,Entire home/apt,5,2.0,3.0,4.0,249.0,3,32,78,7,1,1300147200,1716163200,4.79,4.93,4.89,4.95,4.99,4.62,4.64,0,0.48,1,0,1,1,1,0,1,0,1,1,1,1,0,1,0,0,0,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18920,1173891409488923931,1460073600,within an hour,100.0,48.0,0.0,53.0,1.0,1.0,Ciutat Vella,Entire rental unit,Entire home/apt,2,1.0,1.0,1.0,86.0,32,345,0,0,0,-9223372037,-9223372037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,1,0,0,1,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
18921,1178741129427938140,1589500800,within a few hours,99.0,98.0,0.0,702.0,1.0,1.0,Ciutat Vella,Entire rental unit,Entire home/apt,4,2.0,2.0,2.0,167.0,31,330,0,0,0,-9223372037,-9223372037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,0,1,0,0,0
18922,1178910876458935142,1589500800,within a few hours,99.0,98.0,0.0,702.0,1.0,1.0,Ciutat Vella,Entire rental unit,Entire home/apt,4,2.0,2.0,2.0,165.0,31,330,0,0,0,-9223372037,-9223372037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,0,1,0,0,0
18923,1173984319228839137,1568246400,within an hour,99.0,99.0,0.0,24.0,1.0,1.0,Ciutat Vella,Private room in rental unit,Private room,2,1.0,1.0,1.0,44.0,5,90,0,0,0,-9223372037,-9223372037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


<h3>Importing reviews polarity obtained by sentiment analysis</h3>

In [82]:
sentiment = pd.read_csv('sentiment.csv')

sentiment.head()

,listing_id,sentiment
0,10003263,0.405025
1,1000432235531291341,0.336934
2,1000447810456915898,0.376677
3,1000466852301593784,0.432986
4,1000514588274707195,0.364167


In [83]:
listings['id'] = listings['id'].astype(str)
sentiment['listing_id'] = sentiment['listing_id'].astype(str)

# Merge the datasets on the corresponding ID columns
merged_data = pd.merge(listings, sentiment, left_on='id', right_on='listing_id', how='left')

# Drop the duplicate 'listing_id' column if it exists
merged_data = merged_data.drop(columns=['listing_id', 'id'])

merged_data

,host_since,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_has_profile_pic,host_identity_verified,neighbourhood_group_cleansed,property_type,room_type,accommodates,bathrooms,bedrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,reviews_per_month,wifi,heating,essentials,kitchen,smoke_alarm,air_conditioning,hangers,coffee_maker,dedicated_workspace,hot_water_kettle,hair_dryer,iron,tv,hot_water,washer,dryer,first_aid_kit,fire_extinguisher,refrigerator,pool,Gym,pets_allowed,free_parking_on_premises,paid_parking_on_premises,self_check-in,sentiment
0,1321401600,within an hour,100.0,88.0,0.0,5.0,1.0,1.0,Eixample,Entire loft,Entire home/apt,2,1.0,1.0,1.0,176.0,7,760,90,17,1,1514764800,1716336000,4.74,4.87,4.59,4.83,4.79,4.81,4.67,1,1.15,1,1,1,1,0,0,1,1,1,0,1,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0.428676
1,1321401600,within an hour,98.0,97.0,0.0,36.0,1.0,1.0,Ciutat Vella,Entire rental unit,Entire home/apt,6,1.5,4.0,5.0,74.0,32,760,2,0,0,1467504000,1471046400,4.00,5.00,4.50,5.00,5.00,5.00,4.50,0,0.02,1,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0.265625
2,1263859200,within an hour,95.0,85.0,0.0,41.0,1.0,1.0,Eixample,Entire rental unit,Entire home/apt,8,2.0,3.0,6.0,257.0,1,1125,41,5,0,1369612800,1714608000,4.35,4.45,4.63,4.75,4.68,4.78,4.30,1,0.30,1,1,1,1,0,0,1,1,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0.369518
3,1321747200,NaN,NaN,98.0,1.0,2.0,1.0,1.0,Eixample,Entire rental unit,Entire home/apt,8,NaN,3.0,NaN,NaN,4,1125,414,32,0,1341273600,1710720000,4.92,4.93,4.97,4.94,4.96,4.84,4.80,0,2.84,1,1,1,1,1,1,1,0,0,1,1,1,0,1,1,0,0,1,1,0,0,1,0,0,0,0.439969
4,1268092800,within an hour,100.0,87.0,0.0,6.0,1.0,1.0,Sant Martí,Entire rental unit,Entire home/apt,5,2.0,3.0,4.0,249.0,3,32,78,7,1,1300147200,1716163200,4.79,4.93,4.89,4.95,4.99,4.62,4.64,0,0.48,1,0,1,1,1,0,1,0,1,1,1,1,0,1,0,0,0,1,1,0,0,0,0,0,0,0.367184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18920,1460073600,within an hour,100.0,48.0,0.0,53.0,1.0,1.0,Ciutat Vella,Entire rental unit,Entire home/apt,2,1.0,1.0,1.0,86.0,32,345,0,0,0,-9223372037,-9223372037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,1,0,0,1,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,NaN
18921,1589500800,within a few hours,99.0,98.0,0.0,702.0,1.0,1.0,Ciutat Vella,Entire rental unit,Entire home/apt,4,2.0,2.0,2.0,167.0,31,330,0,0,0,-9223372037,-9223372037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,0,1,0,0,0,NaN
18922,1589500800,within a few hours,99.0,98.0,0.0,702.0,1.0,1.0,Ciutat Vella,Entire rental unit,Entire home/apt,4,2.0,2.0,2.0,165.0,31,330,0,0,0,-9223372037,-9223372037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,0,1,0,0,0,NaN
18923,1568246400,within an hour,99.0,99.0,0.0,24.0,1.0,1.0,Ciutat Vella,Private room in rental unit,Private room,2,1.0,1.0,1.0,44.0,5,90,0,0,0,-9223372037,-9223372037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN


In [84]:
from sklearn.preprocessing import OneHotEncoder

# Identify numeric and categorical columns
numeric_features = merged_data.select_dtypes(include=['int64', 'float64']).columns
categorical_features = merged_data.select_dtypes(include=['object', 'category']).columns

# Encode categorical features
encoder = OneHotEncoder(drop='first', sparse=False)
listings_encoded = pd.DataFrame(encoder.fit_transform(merged_data[categorical_features]), columns=encoder.get_feature_names_out(categorical_features), index=merged_data.index)

# Drop original categorical columns and concatenate the encoded ones
listings = merged_data.drop(columns=categorical_features)
listings = pd.concat([listings, listings_encoded], axis=1)

listings

/Users/krys/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,host_since,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_has_profile_pic,host_identity_verified,accommodates,bathrooms,bedrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,reviews_per_month,wifi,heating,essentials,kitchen,smoke_alarm,air_conditioning,hangers,coffee_maker,dedicated_workspace,hot_water_kettle,hair_dryer,iron,tv,hot_water,washer,dryer,first_aid_kit,fire_extinguisher,refrigerator,pool,Gym,pets_allowed,free_parking_on_premises,paid_parking_on_premises,self_check-in,sentiment,host_response_time_within a day,host_response_time_within a few hours,host_response_time_within an hour,host_response_time_nan,neighbourhood_group_cleansed_Eixample,neighbourhood_group_cleansed_Gràcia,neighbourhood_group_cleansed_Horta-Guinardó,neighbourhood_group_cleansed_Les Corts,neighbourhood_group_cleansed_Nou Barris,neighbourhood_group_cleansed_Sant Andreu,neighbourhood_group_cleansed_Sant Martí,neighbourhood_group_cleansed_Sants-Montjuïc,neighbourhood_group_cleansed_Sarrià-Sant Gervasi,property_type_Boat,property_type_Camper/RV,property_type_Casa particular,property_type_Earthen home,property_type_Entire cabin,property_type_Entire chalet,property_type_Entire condo,property_type_Entire guest suite,property_type_Entire guesthouse,property_type_Entire home,property_type_Entire loft,property_type_Entire place,property_type_Entire rental unit,property_type_Entire serviced apartment,property_type_Entire townhouse,property_type_Entire vacation home,property_type_Entire villa,property_type_Private room,property_type_Private room in bed and breakfast,property_type_Private room in boat,property_type_Private room in casa particular,property_type_Private room in chalet,property_type_Private room in condo,property_type_Private room in cottage,property_type_Private room in dome,property_type_Private room in floor,property_type_Private room in guest suite,property_type_Private room in guesthouse,property_type_Private room in home,property_type_Private room in hostel,property_type_Private room in loft,property_type_Private room in rental unit,property_type_Private room in serviced apartment,property_type_Private room in tiny home,property_type_Private room in townhouse,property_type_Private room in vacation home,property_type_Private room in villa,property_type_Room in aparthotel,property_type_Room in bed and breakfast,property_type_Room in boutique hotel,property_type_Room in hostel,property_type_Room in hotel,property_type_Room in serviced apartment,property_type_Shared room in bed and breakfast,property_type_Shared room in boutique hotel,property_type_Shared room in condo,property_type_Shared room in floor,property_type_Shared room in guest suite,property_type_Shared room in guesthouse,property_type_Shared room in home,property_type_Shared room in hostel,property_type_Shared room in hotel,property_type_Shared room in loft,property_type_Shared room in rental unit,property_type_Shared room in serviced apartment,property_type_Shared room in villa,property_type_Tiny home,property_type_Yurt,room_type_Hotel room,room_type_Private room,room_type_Shared room
0,1321401600,100.0,88.0,0.0,5.0,1.0,1.0,2,1.0,1.0,1.0,176.0,7,760,90,17,1,1514764800,1716336000,4.74,4.87,4.59,4.83,4.79,4.81,4.67,1,1.15,1,1,1,1,0,0,1,1,1,0,1,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0.428676,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1321401600,98.0,97.0,0.0,36.0,1.0,1.0,6,1.5,4.0,5.0,74.0,32,760,2,0,0,1467504000,1471046400,4.00,5.00,4.50,5.00,5.00,5.

In [85]:
from sklearn.preprocessing import StandardScaler

# Scale the numeric data
scaler = StandardScaler()
listings_scaled = scaler.fit_transform(listings)

# Convert the scaled data back to a DataFrame
listings = pd.DataFrame(listings_scaled, columns=listings.columns)

listings

,host_since,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_has_profile_pic,host_identity_verified,accommodates,bathrooms,bedrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,reviews_per_month,wifi,heating,essentials,kitchen,smoke_alarm,air_conditioning,hangers,coffee_maker,dedicated_workspace,hot_water_kettle,hair_dryer,iron,tv,hot_water,washer,dryer,first_aid_kit,fire_extinguisher,refrigerator,pool,Gym,pets_allowed,free_parking_on_premises,paid_parking_on_premises,self_check-in,sentiment,host_response_time_within a day,host_response_time_within a few hours,host_response_time_within an hour,host_response_time_nan,neighbourhood_group_cleansed_Eixample,neighbourhood_group_cleansed_Gràcia,neighbourhood_group_cleansed_Horta-Guinardó,neighbourhood_group_cleansed_Les Corts,neighbourhood_group_cleansed_Nou Barris,neighbourhood_group_cleansed_Sant Andreu,neighbourhood_group_cleansed_Sant Martí,neighbourhood_group_cleansed_Sants-Montjuïc,neighbourhood_group_cleansed_Sarrià-Sant Gervasi,property_type_Boat,property_type_Camper/RV,property_type_Casa particular,property_type_Earthen home,property_type_Entire cabin,property_type_Entire chalet,property_type_Entire condo,property_type_Entire guest suite,property_type_Entire guesthouse,property_type_Entire home,property_type_Entire loft,property_type_Entire place,property_type_Entire rental unit,property_type_Entire serviced apartment,property_type_Entire townhouse,property_type_Entire vacation home,property_type_Entire villa,property_type_Private room,property_type_Private room in bed and breakfast,property_type_Private room in boat,property_type_Private room in casa particular,property_type_Private room in chalet,property_type_Private room in condo,property_type_Private room in cottage,property_type_Private room in dome,property_type_Private room in floor,property_type_Private room in guest suite,property_type_Private room in guesthouse,property_type_Private room in home,property_type_Private room in hostel,property_type_Private room in loft,property_type_Private room in rental unit,property_type_Private room in serviced apartment,property_type_Private room in tiny home,property_type_Private room in townhouse,property_type_Private room in vacation home,property_type_Private room in villa,property_type_Room in aparthotel,property_type_Room in bed and breakfast,property_type_Room in boutique hotel,property_type_Room in hostel,property_type_Room in hotel,property_type_Room in serviced apartment,property_type_Shared room in bed and breakfast,property_type_Shared room in boutique hotel,property_type_Shared room in condo,property_type_Shared room in floor,property_type_Shared room in guest suite,property_type_Shared room in guesthouse,property_type_Shared room in home,property_type_Shared room in hostel,property_type_Shared room in hotel,property_type_Shared room in loft,property_type_Shared room in rental unit,property_type_Shared room in serviced apartment,property_type_Shared room in villa,property_type_Tiny home,property_type_Yurt,room_type_Hotel room,room_type_Private room,room_type_Shared room
0,-1.048200,0.400321,0.130683,-0.499568,-0.392745,0.154649,0.302458,-0.617899,-0.484762,-0.650328,-0.581749,-0.062137,-0.262852,0.513224,0.448550,0.213141,-0.031861,0.564429,0.587232,0.296577,0.471450,-0.051135,0.234169,0.140055,0.160113,0.377518,1.318479,-0.184237,0.287409,0.804683,0.524498,0.339803,-0.590429,-1.048798,0.611853,1.145494,1.095244,-0.756549,0.610141,0.651731,0.811293,0.623773,0.798636,-0.408286,-0.538397,-0.542478,0.744848,-0.148066,-0.112614,-0.433773,-0.16149,-0.307203,-0.48534,0.161335,-0.292762,-0.477930,0.885740,-0.424969,1.348606,-0.301607,-0.174782,-0.147879,-0.109937,-0.130311,-0.319

<h3>Handling missing values through Regression Imputation</h3>

In [86]:
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer

# Function to fill missing values using linear regression
def fill_missing_values(df):
    df_filled = df.copy()
    
    for column in df.columns:
        if df[column].isnull().sum() > 0:

            # Separate the data into columns with and without missing values
            df_not_null = df_filled[df_filled[column].notnull()]
            df_null = df_filled[df_filled[column].isnull()]
            
            if df_not_null.empty or df_null.empty:
                # Skip column if there are no missing values
                continue
            
            # Define the model
            model = LinearRegression()
            
            # Ensure that X_train and X_test have no missing values
            X_train = df_not_null.drop(columns=[column])
            y_train = df_not_null[column]
            
            # Check for any NaNs in the training data
            if X_train.isnull().any().any():
                # If training data for the column contains NaNs, imput missing values before training
                imputer = SimpleImputer(strategy='mean')
                X_train = imputer.fit_transform(X_train)
            
            X_test = df_null.drop(columns=[column])
            
            if X_test.isnull().any().any():
                # If test data for the column contains NaNs, imput missing values before prediction
                X_test = imputer.transform(X_test)
            
            # Train the model
            model.fit(X_train, y_train)
            
            # Predict the missing values
            predicted_values = model.predict(X_test)
            
            # Assign the predicted values to the original dataframe
            df_filled.loc[df_filled[column].isnull(), column] = predicted_values
    
    return df_filled


# Apply the function to fill missing values in the numeric data
listings_filled = fill_missing_values(listings)

listings_filled

,host_since,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_has_profile_pic,host_identity_verified,accommodates,bathrooms,bedrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,reviews_per_month,wifi,heating,essentials,kitchen,smoke_alarm,air_conditioning,hangers,coffee_maker,dedicated_workspace,hot_water_kettle,hair_dryer,iron,tv,hot_water,washer,dryer,first_aid_kit,fire_extinguisher,refrigerator,pool,Gym,pets_allowed,free_parking_on_premises,paid_parking_on_premises,self_check-in,sentiment,host_response_time_within a day,host_response_time_within a few hours,host_response_time_within an hour,host_response_time_nan,neighbourhood_group_cleansed_Eixample,neighbourhood_group_cleansed_Gràcia,neighbourhood_group_cleansed_Horta-Guinardó,neighbourhood_group_cleansed_Les Corts,neighbourhood_group_cleansed_Nou Barris,neighbourhood_group_cleansed_Sant Andreu,neighbourhood_group_cleansed_Sant Martí,neighbourhood_group_cleansed_Sants-Montjuïc,neighbourhood_group_cleansed_Sarrià-Sant Gervasi,property_type_Boat,property_type_Camper/RV,property_type_Casa particular,property_type_Earthen home,property_type_Entire cabin,property_type_Entire chalet,property_type_Entire condo,property_type_Entire guest suite,property_type_Entire guesthouse,property_type_Entire home,property_type_Entire loft,property_type_Entire place,property_type_Entire rental unit,property_type_Entire serviced apartment,property_type_Entire townhouse,property_type_Entire vacation home,property_type_Entire villa,property_type_Private room,property_type_Private room in bed and breakfast,property_type_Private room in boat,property_type_Private room in casa particular,property_type_Private room in chalet,property_type_Private room in condo,property_type_Private room in cottage,property_type_Private room in dome,property_type_Private room in floor,property_type_Private room in guest suite,property_type_Private room in guesthouse,property_type_Private room in home,property_type_Private room in hostel,property_type_Private room in loft,property_type_Private room in rental unit,property_type_Private room in serviced apartment,property_type_Private room in tiny home,property_type_Private room in townhouse,property_type_Private room in vacation home,property_type_Private room in villa,property_type_Room in aparthotel,property_type_Room in bed and breakfast,property_type_Room in boutique hotel,property_type_Room in hostel,property_type_Room in hotel,property_type_Room in serviced apartment,property_type_Shared room in bed and breakfast,property_type_Shared room in boutique hotel,property_type_Shared room in condo,property_type_Shared room in floor,property_type_Shared room in guest suite,property_type_Shared room in guesthouse,property_type_Shared room in home,property_type_Shared room in hostel,property_type_Shared room in hotel,property_type_Shared room in loft,property_type_Shared room in rental unit,property_type_Shared room in serviced apartment,property_type_Shared room in villa,property_type_Tiny home,property_type_Yurt,room_type_Hotel room,room_type_Private room,room_type_Shared room
0,-1.048200,4.003205e-01,0.130683,-0.499568,-0.392745,0.154649,0.302458,-0.617899,-0.484762,-0.650328,-0.581749,-0.062137,-0.262852,0.513224,0.448550,0.213141,-0.031861,0.564429,0.587232,0.296577,0.471450,-0.051135,0.234169,0.140055,0.160113,0.377518,1.318479,-0.184237,0.287409,0.804683,0.524498,0.339803,-0.590429,-1.048798,0.611853,1.145494,1.095244,-0.756549,0.610141,0.651731,0.811293,0.623773,0.798636,-0.408286,-0.538397,-0.542478,0.744848,-0.148066,-0.112614,-0.433773,-0.16149,-0.307203,-0.48534,0.161335,-0.292762,-0.477930,0.885740,-0.424969,1.348606,-0.301607,-0.174782,-0.147879,-0.109937,-0.130311,-0

<h3>Feature selection with Lasso Regularization</h3>

In [87]:
#from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# Separate features and target
#X = listings.drop(columns=["price"])
#y = listings["price"]

# Split data into training and test sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Standardize features
#scaler = StandardScaler()
#X_train_scaled = scaler.fit_transform(X_train)
#X_test_scaled = scaler.transform(X_test)

# Initialize and fit Lasso model
#lasso = Lasso(alpha=0.01)  # alpha is the regularization strength
#lasso.fit(X_train_scaled, y_train)

# Make predictions
#y_pred = lasso.predict(X_test_scaled)#

# Evaluate the model
#mse = mean_squared_error(y_test, y_pred)
#print(f"Mean Squared Error: {mse}")

# View the coefficients
#print("Lasso Coefficients:", lasso.coef_)

In [88]:
import numpy as np

# Get the feature names if you have them
#feature_names = np.array(X.columns)  # if X is a DataFrame

# Get the coefficients from the Lasso model
#coefficients = lasso.coef_

# Identify the features with non-zero coefficients
#selected_features = feature_names[coefficients != 0]

#print("Selected Features:", selected_features)

In [89]:
#X_selected = X[selected_features]

#print("Number of features in updated dataset:", X_selected.shape[1])


In [90]:
#X_selected.to_csv('X.csv', index=False)

#y.to_csv('y.csv', index=False)

In [91]:
listings_filled.to_csv('final_listings.csv')